# 第一部分：统计不同药物在每个数据集中的相似度

**相似度计算方法**
- Target相似度：
- Relation相似度：
- Triple相似度：
<br/>

**后来发现代码中已经给出了一种相似度计算方法，但不知道作者为什么要算相似度**

In [1]:
import pandas as pd
import numpy as np

import sqlite3
import csv
# from sklearn.metrics import accuracy_score
# from pytorchtools import EarlyStopping
from collections import defaultdict
import os
import random

import debugUtiles as db

# seed = 0

random.seed(1)
os.environ['PYTHONHASHSEED'] = str(1)
np.random.seed(1)


'''
输入: 药物id字典, 数据集编号
输出: 知识图谱数据集, 尾节点长度, 关系长度
'''
def read_dataset(drug_name_id,num):
    # kg: key: drug_id, value: list of (tail_id, relation_id)
    kg = defaultdict(list)
    # tails: key: tail_name, value: tail_id
    tails = {}
    # relations: key: relation_name, value: relation_id 
    relations = {}
    
    drug_list=[]
    filename = "../dataset/dataset"+str(num)+".txt"
    with open(filename, encoding="utf8") as reader:
        for line in reader:
            string= line.rstrip().split('//',2)
            head=string[0]
            tail=string[1]
            relation=string[2]
            drug_list.append(drug_name_id[head])
            if tail not in tails:
                tails[tail] = len(tails)
            if relation not in relations:
                relations[relation] = len(relations)
            if num==3:
                kg[drug_name_id[head]].append((drug_name_id[tail], relations[relation]))
                kg[drug_name_id[tail]].append((drug_name_id[head], relations[relation]))
            else:
                kg[drug_name_id[head]].append((tails[tail], relations[relation]))
    
    # db.log(kg)
    return kg,len(tails),len(relations)

def prepare(mechanism, action):
    d_label = {}
    d_event = []
    new_label = []
    # 将 机制 和 作用行为 合并 为 事件
    for i in range(len(mechanism)):
        d_event.append(mechanism[i] + " " + action[i])
    # 统计每个 事件 出现的次数，输出为字典，key为 事件，value为次数，并从大到小排序
    count = {}
    for i in d_event:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    list1 = sorted(count.items(), key=lambda x: x[1], reverse=True)
    # list1 like [('mechanism action', 10), ('mechanism action', 8), ('mechanism action', 5)]
    
    # 得到每项不同事件的标签
    for i in range(len(list1)):
        d_label[list1[i][0]] = i
    
    # 将所有事件数据转换为标签
    for i in range(len(d_event)):
        new_label.append(d_label[d_event[i]])

    # 返回事件标签和事件类别的个数
    # db.log('envent num: '+str(len(count)))
    # db.log(new_label)
    return new_label,len(count)

def main():
    conn = sqlite3.connect("../dataset/event.db")
    
    # 药物数据
    df_drug = pd.read_sql('select * from drug;', conn)
    
    # 药物相互作用数据
    extraction = pd.read_sql('select * from extraction;', conn)
    
    # 提取药物相互作用数据的机制、作用行为、药物A、药物B
    mechanism = extraction['mechanism']
    action = extraction['action']
    drugA = extraction['drugA']
    drugB = extraction['drugB']
    
    # 得到interaction数据标签和interaction类别的个数
    # newlabel: 所有interaction数据标签
    new_label,event_num = prepare(mechanism, action)
    new_label = np.array(new_label)

    # db.log('shape of new_label: '+str(new_label.shape))
    
    # 将每类药物的name转换为id，dict1是一个药物id字典
    dict1 = {}
    for i in df_drug["name"]:
        dict1[i] = len(dict1)
    
    # db.log(dict1)
    
    # 将所有药物、药物A、药物B转换为id标签
    drug_name = [dict1[i] for i in df_drug["name"]]
    
    drugA_id = [dict1[i] for i in drugA]
    drugB_id = [dict1[i] for i in drugB]
    dataset1_kg, dataset1_tail_len, dataset1_relation_len = read_dataset(dict1,1)
    dataset2_kg, dataset2_tail_len, dataset2_relation_len = read_dataset(dict1,2)
    dataset3_kg, dataset3_tail_len, dataset3_relation_len = read_dataset(dict1,3)
    dataset4_kg, dataset4_tail_len, dataset4_relation_len = read_dataset(dict1,4)
    x_datasets = {"drugA": drugA_id, "drugB": drugB_id}
    x_datasets = pd.DataFrame(data=x_datasets)
    
    # x_datasets like: [[drugA_id1, drugB_id1], [drugA_id2, drugB_id2], ...]
    x_datasets = x_datasets.to_numpy()
    # db.log(x_datasets[0][0])
    
    # dataset_kg1 like: {drug_id1: [(tail_id1, relation_id1), (tail_id2, relation_id2), ...], ...}
    dataset={}
    dataset["dataset1"],dataset["dataset2"],dataset["dataset3"],dataset["dataset4"] = dataset1_kg,dataset2_kg,dataset3_kg,dataset4_kg
    tail_len={}
    tail_len["dataset1"],tail_len["dataset2"],tail_len["dataset3"],tail_len["dataset4"] = dataset1_tail_len,dataset2_tail_len,dataset3_tail_len,dataset4_tail_len
    relation_len={}
    relation_len["dataset1"],relation_len["dataset2"],relation_len["dataset3"],relation_len["dataset4"] = dataset1_relation_len,dataset2_relation_len,dataset3_relation_len,dataset4_relation_len

    temp_kg = [defaultdict(list) for i in range(4)]
    for p,kg in enumerate(dataset):
        for i in dataset[kg].keys():
            for j in dataset[kg][i]:
                temp_kg[p][i].append(j[0])
    # temp_kg[0] like: [{drug_id1: [tail_id1, tail_id2, ...], ...}, ...]
    # db.log(temp_kg[0])
    
    # 572 is here db.log('drug_num: '+str(len(dict1)))

    # db.log(dataset4_kg)

    # 统计每类interaction中药物A和药物B的数量
    # temp_drugA like: [[drugA_id1, drugA_id2, ...], ...], where the 0 dim is the interaction class
    temp_drugA = [[] for i in range(event_num)]
    temp_drugB = [[] for i in range(event_num)]
    for i in range(len(new_label)):
        temp_drugA[new_label[i]].append(drugA_id[i])
        temp_drugB[new_label[i]].append(drugB_id[i])

    # for i in range(event_num):
    #     if len(temp_drugA[i]) != len(temp_drugB[i]):
    #         db.log('error: drugA num != drugB num')
        # db.log('event '+str(i)+' drugA num: '+str(len(temp_drugA[i]))+' drugB num: '+str(len(temp_drugB[i])))
    # db.log(temp_drugA)
    
    # 作用：将药物类别集合分为5个子类，用于5折交叉验证
    drug_cro_dict = {}
    for i in range(event_num):
        for j in range(len(temp_drugA[i])):
            drug_cro_dict[temp_drugA[i][j]] = j % 5
            drug_cro_dict[temp_drugB[i][j]] = j % 5
    
    # db.log('drug_cro_dict: '+str(drug_cro_dict))
    
    train_drug = [[] for i in range(5)]
    test_drug = [[] for i in range(5)]
    # 将全部数据分为test data 和 train data,这种划分方式很奇怪
    for i in range(5):
        for dr_key in drug_cro_dict.keys():
            if drug_cro_dict[dr_key] == i:
                test_drug[i].append(dr_key)
            else:
                train_drug[i].append(dr_key)

    # db.log('4/5 drug_name: '+str(len(drug_name) * 0.8))
    # db.log('train_drug: '+str(len(train_drug[0])))
    
    for cross_ver in range(5):
        X_train = []
        X_test = []
        y_train = []
        y_test = []
        
        X_testAnot = []
        X_testBnot = []
        for i in range(len(drugA)):
            if (drugA_id[i] in np.array(train_drug[cross_ver])) and (drugB_id[i] in np.array(train_drug[cross_ver])):
                X_train.append(i)
                y_train.append(i)
            if (drugA_id[i] not in np.array(train_drug[cross_ver])) and (drugB_id[i] in np.array(train_drug[cross_ver])):
                X_test.append(i)
                y_test.append(i)
                X_testAnot.append(i)
            # Task2 任务定义
            if (drugA_id[i] in np.array(train_drug[cross_ver])) and (drugB_id[i] not in np.array(train_drug[cross_ver])):
                X_test.append(i)
                y_test.append(i)
                X_testBnot.append(i)

        # x_datasets是DDIML数据集中的所有行
        # train_x = x_datasets[X_train]   # 药物标签对
        # train_y = new_label[y_train]    # interaction的标签
        
        test_x = x_datasets[X_test]
        # test_y = new_label[y_test]
        
        # test_xAnot: 药物A不在训练集中，药物B在训练集中
        # test_xBnot: 药物A在训练集中，药物B不
        test_xAnot = x_datasets[X_testAnot]
        test_xBnot = x_datasets[X_testBnot]
        test_xnot = [i[0] for i in test_xAnot] + [i[1] for i in test_xBnot]
        test_xnot_set = set(test_xnot)
        train_x_set = set([i for i in range(572) if i not in test_xnot_set])  # 572 is the number of drug names
        
        
        # print("test_x len: ", len(test_x))
        print("test_xAnot len: ", len(test_xAnot))
        print("test_xBnot len: ", len(test_xBnot))
        print("sum of test_xAnot and test_xBnot: ", len(test_xAnot) + len(test_xBnot))
        
        
        # 这种划分方式对于数据集的划分也不是均匀的，只是说是五种类别中选一种类别
        # db.log("Shape of train_x: "+str(train_x.shape))     # 25547
        # db.log("Shape of test_x: "+str(test_x.shape))     # 10619
        
        # dataset1_kg dataset2_kg dataset3_kg dataset4_kg dataset
        for kg_name in dataset:
            
            if kg_name == "dataset4":
                break
            
            # whole_kg: 包含了所有药物的知识图谱
            # kg: 只包含了训练集中的药物的知识图谱
            whole_kg = dataset[kg_name]
            kg = {i:whole_kg[i] for i in train_x_set}
            
            tar_scores = defaultdict(dict)
            rela_scores = defaultdict(dict)
            tr_scores = defaultdict(dict)
            for notin_drug_id in test_xnot_set:
                tar_row_scores, rela_row_scores, tr_row_scores = computeSimilarityOnIOU(notin_drug_id, kg, whole_kg)
                tar_scores[notin_drug_id] = tar_row_scores
                rela_scores[notin_drug_id] = rela_row_scores
                tr_scores[notin_drug_id] = tr_row_scores
            
            tar_scores = sorted(tar_scores.items(), key=lambda x: x[0])
            rela_scores = sorted(rela_scores.items(), key=lambda x: x[0])
            tr_scores = sorted(tr_scores.items(), key=lambda x: x[0])
            
            # 检查并创建文件夹
            output_dir = 'inspectDataset'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # 保存 tar_scores 到 CSV 文件
            tar_file_path = os.path.join(output_dir, f'{kg_name}_tar_scores_cross{cross_ver}.csv')
            with open(tar_file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['药物ID'] + list(train_x_set))  # 添加表头
                for drug_id, scores in tar_scores:
                    writer.writerow([drug_id] + [scores[i] for i in sorted(scores.keys())])
            
            # 保存 rela_scores 到 CSV 文件
            rela_file_path = os.path.join(output_dir, f'{kg_name}_rela_scores_cross{cross_ver}.csv')
            with open(rela_file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['药物ID'] + list(train_x_set))  # 添加表头
                for drug_id, scores in rela_scores:
                    writer.writerow([drug_id] + [scores[i] for i in sorted(scores.keys())])
            
            # 保存 tr_scores 到 CSV 文件
            tr_file_path = os.path.join(output_dir, f'{kg_name}_tr_scores_cross{cross_ver}.csv')
            with open(tr_file_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['药物ID'] + list(train_x_set))  # 添加表头
                for drug_id, scores in tr_scores:
                    writer.writerow([drug_id] + [scores[i] for i in sorted(scores.keys())])
            
            print("largest similarities of targets in {} is: {}".format(kg_name, {scores_tuple[0]:max([scores[i] for i in scores_tuple[1].keys()]) for scores_tuple in tar_scores}))
            print("largest similarities of relations in {} is: {}".format(kg_name, {scores_tuple[0]:max([scores[i] for i in scores_tuple[1].keys()]) for scores_tuple in rela_scores}))
            print("largest similarities of triples in {} is: {}".format(kg_name, {scores_tuple[0]:max([scores[i] for i in scores_tuple[1].keys()]) for scores_tuple in tr_scores}))

'''
input: drug_id(在测试集中的药物的id), kg(只包含训练集药物的知识图谱), whole_kg(完成的知识图谱)
output: 三个相似度分数(目标、关系、三元组)
'''
def computeSimilarityOnIOU(drug_id, kg, whole_kg):
    drt_triples = whole_kg[drug_id]
    tar_row_scores = {}
    rela_row_scores = {}
    tr_row_scores = {}
    
    tar_kg_set = {i:set([tri[0] for tri in kg[i]]) for i in kg}
    rela_kg_set = {i:set([tri[1] for tri in kg[i]]) for i in kg}
    tr_kg_set = {i:set(kg[i]) for i in kg}
    
    tar_drt_set = set([i[0] for i in drt_triples])
    rela_drt_set = set([i[1] for i in drt_triples])
    tr_drt_set = set(drt_triples)
    
    # 计算交并比相似度分数（0-1）
    for i in tar_kg_set:
        tar_row_scores[i] = len(tar_kg_set[i].intersection(tar_drt_set)) / len(tar_kg_set[i].union(tar_drt_set))
        rela_row_scores[i] = len(rela_kg_set[i].intersection(rela_drt_set)) / len(rela_kg_set[i].union(rela_drt_set))
        tr_row_scores[i] = len(tr_kg_set[i].intersection(tr_drt_set)) / len(tr_kg_set[i].union(tr_drt_set))
    
    # tar_row_scores[drug_id] = -1
    # rela_row_scores[drug_id] = -1
    # tr_row_scores[drug_id] = -1
    # for i in kg:
    #     for j in kg[i]:
    #         j_tar_set = set([k[0] for k in j])
    #         for k in drt_triples:
    #             # print("j: {}, k: {}".format(j, k))
    #             if j[0] == k[0]:
    #                 tar_row_scores[i] += 1
    #             if j[1] == k[1]:
    #                 rela_row_scores[i] += 1
    #             if j == k:
    #                 tr_row_scores[i] += 1
    
    
    # tar_row_scores = np.array([ins / (len(kg[i]) + len(drt_triples) - ins) for i, ins in enumerate(tar_row_scores)])
    # rela_row_scores = np.array([ins / (len(kg[i]) + len(drt_triples) - ins) for i, ins in enumerate(rela_row_scores)])
    # tr_row_scores = np.array([ins / (len(kg[i]) + len(drt_triples) - ins) for i, ins in enumerate(tr_row_scores)])

    return tar_row_scores, rela_row_scores, tr_row_scores

if __name__ == '__main__':
    main()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,